## Pre-trained Net used on a new set of classes


Sample based on "Fine-tune InceptionV3 on a new set of classes" from https://keras.io/applications/  
Upload in a directory images for training, a directory for each class  
Upload in a separate directory images for validation, a directory for each class  
Upload test images in main directory  

First: Install **Keras** on environment  

### Pre-requisites library

In [1]:
! pip install keras
! pip install pillow
! pip install Flask
! pip install Flask-Uploads
! pip install Flask-Uploads --upgrade

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: Flask-Uploads in /usr/local/lib/python2.7/dist-packages
Requirement already up-to-date: Flask>=0.8.0 in /usr/local/lib/python2.7/dist-packages (from Flask-Uploads)
Requirement already up-to-date: Jinja2>=2.10 in /usr/local/lib/python2.7/dist-packages (from Flask>=0.8.0->Flask-Uploads)
Requirement already up-to-date: itsdangerous>=0.24 in /usr/local/lib/python2.7/dist-packages (from Fla

In [2]:
! pwd

/shared/DL-workshop


In [3]:
import warnings
warnings.filterwarnings('ignore')

Check Tensorflow release

In [4]:
import tensorflow as tf; print(tf.__version__)

1.7.0


In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
import PIL

Using TensorFlow backend.


### Setup & data preparation

In [6]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir =  'data/training' #contains classes to train
validation_data_dir = 'data/test' #contains classes for validation

nb_train_samples = 779
nb_validation_samples = 800
nb_epoch = 1
nb_classes  = 5
model_name = "inception_retrained_model1"

# Data for Training and Validation
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255)#,
 #       shear_range=0.2,
 #       zoom_range=0.2,
 #       horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='categorical'
)

label_map = (train_generator.class_indices)

print ("\nLabel map in training directory:")
print label_map

Found 779 images belonging to 5 classes.

Label map in training directory:
{'portiera': 1, 'ruote': 2, 'specchi': 3, 'paraurti': 0, 'vetri': 4}


### Training step: go to Load model re-trained cell to skip training phase

In [ ]:

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='categorical'
)



nb_classes =  len(label_map)

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(nb_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])



print "start history model"
history = model.fit_generator(
    train_generator,
    nb_epoch=nb_epoch,
    samples_per_epoch=128,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples) 

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
print("base_model")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
#for layer in model.layers[:172]:
for layer in model.layers[:300]:
    layer.trainable = False
#for layer in model.layers[172:]:
for layer in model.layers[300:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
# model.fit_generator(...)
# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples,
        steps_per_epoch=10)

model.save(model_name+".h5")

In [ ]:
print("predictions")
print(predictions)

### Conversion Net+weights to .pb Tensorflow protobuf format

In [ ]:
from keras.models import load_model
import keras.backend as K
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.training import saver as saver_lib

def convert_keras_to_pb(keras_model, out_names, models_dir, model_filename):
    model = load_model(keras_model+".h5")
    print(model.summary())
    K.set_learning_phase(0)
    sess = K.get_session()
    saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
    checkpoint_path = saver.save(sess, './saved_ckpt', global_step=0, latest_filename='checkpoint_state')
    graph_io.write_graph(sess.graph, '.', 'tmp.pb')
    freeze_graph.freeze_graph('./tmp.pb', '',
                              False, checkpoint_path, out_names,
                              "save/restore_all", "save/Const:0",
                              models_dir+model_filename, False, "")

model_name = "inception_retrained_model1"

convert_keras_to_pb(model_name,"dense_4",".","trained.pb")

### Load model re-trained

In [7]:
model_name = "inception_retrained_model1"
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
import numpy

# returns a compiled model
# identical to the previous one
model = load_model(model_name+".h5")



### Inference function

In [8]:
import json
def inference(imagepath):
    img_path = imagepath
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    preds = model.predict(x)
    
    # Decode results
    inv_map = {v: k for k, v in label_map.iteritems()}
    i=0
    a = []
    for prob in preds[0]:
        a.append([i,int(prob*100)])
        i+=1

    sorted= numpy.argsort(a,axis=0)
    final =[]
    for elem in sorted:
        final.append([inv_map[elem[1]],int(preds[0][elem[1]]*100)])
    inference= numpy.flip(final,axis=0)
    
    partJson= '{ \"name\":\"'+imagepath+'\",  \"results\": [ '
  
    for i in range(0,5):
        partJson+='{ \"class' +str(i+1) + '\":\"' + str(inference[i][0]) + '\", \"value\":\"' + str(inference[i][1]) + '\" },' 
    json = partJson[:-1]+ ']  }'
    return json

print("Test on a local image:")
result = inference('wheel.jpg')
print (result)

Test on a local image:
{ "name":"wheel.jpg",  "results": [ { "class1":"ruote", "value":"52" },{ "class2":"vetri", "value":"12" },{ "class3":"portiera", "value":"12" },{ "class4":"paraurti", "value":"11" },{ "class5":"specchi", "value":"10" }]  }


## REST Server for inference

In [ ]:
import os
from flask import Flask, request, redirect, url_for,Response
from werkzeug.utils import secure_filename

UPLOAD_FOLDER = os.getcwd()+'/img'
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

#app = Flask(__name__)
app = Flask(os.getcwd())

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def root_dir():  # pragma: no cover
    #print('TEST')
    #return os.path.abspath(os.path.dirname(__file__))
    return os.getcwd()


def get_file(filename):  # pragma: no cover
    try:

        src = os.path.join(root_dir(), filename)
        # Figure out how flask returns static files
        # Tried:
        # - render_template
        # - send_file
        # This should not be so non-obvious
        return open(src).read()
    except IOError as exc:
        return str("NOT_FOUND")

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        print(root_dir())
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit a empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            #-----------------------------------------------------------------------------
            # INFERENCE
            result = inference(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            
            #-----------------------------------------------------------------------------
            
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            open(os.path.join(app.config['UPLOAD_FOLDER'], filename+'.ck'),'w').close()
            #return redirect(url_for('upload_file',filename=filename))
            return result

    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=file>
         <input type=submit value=Upload>
    </form>
    getfile: http://[hostname]:6006/inference?filename=filesent
    '''

@app.route('/inference', methods=['GET'])
def metrics():  # pragma: no cover
    fileResponse=request.args.get('filename', '')
    content = get_file(UPLOAD_FOLDER+"/"+fileResponse+".json")
    print(UPLOAD_FOLDER+"/"+fileResponse+".json")
    return Response(content, mimetype="application/json")

if __name__ == '__main__':
    print(root_dir())
    #app.run(debug=True)
    app.run(host='0.0.0.0',port=6006)

/shared/DL-workshop
 * Serving Flask app "/shared/DL-workshop" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:6006/ (Press CTRL+C to quit)
0.0.0.0 - - [22/May/2018 18:24:30] "GET / HTTP/1.1" 200 -
0.0.0.0 - - [22/May/2018 18:24:46] "POST / HTTP/1.1" 200 -


/shared/DL-workshop
